# Quick demonstration of vortex dynamics

Here are to examples of Ubermag notebooks of whett your appetite towards the metapackage and it's riveting possibilities. First, let's start by importing the neccesary tools. The following three lines will be at the beginning of every ubermag simulation.

In [1]:
import oommfc as oc
import discretisedfield as df
import micromagneticmodel as mm

%matplotlib inline

## Vortex dynamics in a disk
Here, we will study how does a vortex move in a disk with exchange coupling only. For the initial conditions, we will set up the magnetization of the disk such that it starts with a vortex. Then, we will the system evolve in such a manner that it will minimize it's energy and we shall see how do the different domains behave.

First off, the parameters of the system. For brevity, I shall assume some familiarity with mesh simulations, energy terms in magnetic systems, and the Landau-Lifshitz-Gilbert equation. For a detailed treatment, I recommend reading [C. Albert, 2019](https://arxiv.org/pdf/1810.12365.pdf). 

In [2]:
# Parameters
R = 50e-9 # disk radius (m)
d = 5e-9 # discretization cell size (i.e. the length of the edge of our cubic mesh cells) (m)
Ms = 8e5 # saturation magnetization for the material (A/m)
A = 13e-12 # exchange energy constant (J/m)
gamma0= 2.211e5 # gyromagnetic ratio (m/As)
alpha = 0.2 # Gilbert damping constant

Now, we define the mesh (i.e. the regions of material that we can approximate to have a single magnetic domain each), In this case, we have a parallelepiped region of square face of edge 2R and thickness d.

In [3]:
# Mesh
p1=(-R, -R, -d/2)
p2=(R, R, d/2)
cell=(d, d, d)
region=df.Region(p1=p1,p2=p2)
mesh=df.Mesh(region=region, cell=cell)

Next, we will define a `system` object that will contain the mesh, its energy and dynamics. To that system, we can assign the specific mechanisms we have in mind for out simulations. In this case, our disk will only have exchange energy and it will evolve with precession and damping.

In [4]:
# Definición del sistema micromagnético

system = mm.System(name='disk_with_vortex')
system.energy = mm.Exchange(A=A) + mm.Demag()
system.dynamics = mm.Precession(gamma0=gamma0) + mm.Damping(alpha=alpha)

A nice feature of Ubermag is that it can actually produce human-readable equations, so we can check the mechanism present in our simulation at any time.

In [5]:
system.energy

Exchange(A=1.3e-11) + Demag()

As inputted before, the only mechanisms are exchange interaction and stray field demagnetization. For the dynamics, we write:

In [6]:
system.dynamics

Precession(gamma0=221100.0) + Damping(alpha=0.2)

Just precession and damping terms. Now, for the geometry and the initial conditions of the system, we must define functions that return the actual object we are looking to simulate. It is easy to see that we have chosen a disk for the former and a vectorial field of uniform curl for the latter.

In [7]:
def geometry(pos):
    x, y, z = pos
    if (x)**2 + (y)**2 < R**2:
        return Ms
    else:
        return 0
    
def m_initial(pos):
    x, y, z= pos
    return (-1e9*y, 1e9*x, 10)

Finally, we input the mesh, dimensions, initial conditions and geometry into the system. Then, we can visualize the results.

In [8]:
system.m = df.Field(mesh, dim=3, value=m_initial, norm=geometry)
system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)
system.m.k3d_vector(color_field=system.m.z, head_size=10)

Output()

Output()

Now, we relax the system and let it evolve on its own. We see that the curvature of the skyrmion increases

In [9]:
md = oc.MinDriver()
md.drive(system)
system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)
system.m.k3d_vector(color_field=system.m.z, head_size=10)

Running OOMMF (TclOOMMFRunner) [2021/04/13 16:16]... (6.9 s)


Output()

Output()

Finally, we can apply an external magnetic field (via the Zeeman energy) and see that the vortex will move in that direction.

In [10]:
system.energy += mm.Zeeman(H=(1e4, 0, 0))
md.drive(system)
system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)
system.m.k3d_vector(color_field=system.m.z, head_size=10)

Running OOMMF (TclOOMMFRunner) [2021/04/13 16:16]... (7.1 s)


Output()

Output()

## Domain wall pair through a junction
Picking the right parameters and geomtry, we can observe a transition from wall domain pair to vortex when the pair goes through a junction of different channel widths.

In [11]:
# Parameters
L = 150e-9 # length (m)
w = 50e-9 # width (m)
d = 2e-9 # size of the discretization cell, also thickness (m)
Ms = 5.8e5 # saturation (A/m)
A = 15e-12 # exchange enery constant (J/m)
D = 3e-3 # Dzyaloshinskii-Moriya energy constant (J/m**2)
K = 0.5e6 # uniaxial anisotropy constant (J/m**3)
u = (0, 0, 1) # easy axis
gamma0= 2.211e5 # gyromagnetic ratio (m/As)
alpha = 0.3 # Gilbert damping

The mesh is a parallelepiped in this case with edges L, w, and d

In [12]:
# Mesh
p1=(0, 0, 0)
p2=(L, w, d)
cell=(d, d, d)
region=df.Region(p1=p1,p2=p2)
mesh=df.Mesh(region=region, cell=cell)

In this case, we must include the Dzyaloshinskii-Moriya interaction and uniaxial anisotropy in order for the vortices to appear from curl-trivial initial conditions. The dynamics remain the same

In [13]:
system = mm.System(name='domain_wall_conversion_thin_film')
system.energy = mm.Exchange(A=A) + \
                mm.DMI(D=D, crystalclass='Cnv') + \
                mm.UniaxialAnisotropy(K=K, u=u)
system.dynamics = mm.Precession(gamma0=gamma0) + mm.Damping(alpha=alpha)

Check out the new energy terms

In [14]:
system.energy

Exchange(A=1.5e-11) + DMI(D=0.003, crystalclass='Cnv') + UniaxialAnisotropy(K=500000.0, u=(0, 0, 1))

Now, for the geometry and the initial conditions. Since we are working with a plate, we can obviate the z component. We are looking for something like this:

![Thinned-out plate for skyrmion production](images/SkyrmionGeometry.png)

The literal functional form of that is:
 $$ y(x)=   \left\{
\begin{array}{ll}
      \tfrac{3}{4}w H(x+l) - \tfrac{1}{4}w H(x+l) & x < l \\
      w H(x-l) & x \geqslant l \\
\end{array} 
\right.  $$

There is, however a much easier way to codify the geometry. We just have to return zero when there is not supposed to be material, and the magnetization when it is. That can be easily achieved with a conditional. The initial conditions are given such that we have a well-defined negative monodomain in a positve environment. 

In [15]:
def geometry(pos):
    x, y, z = pos
    if x < 50e-9 and (y < 15e-9 or y > 35e-9):
        return 0
    else:
        return Ms
    
def m_initial(pos):
    x, y, z= pos
    if (x>10e-9) and (x<=30e-9):
        return (0, 0, -1)
    else:
        return (0, 0, 1)

system.m = df.Field(mesh, dim=3, value=m_initial, norm=geometry)
system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)

Output()

When we relax the system, the negative strip enlarges, but it does stop at the width junction.

In [16]:
md = oc.MinDriver()
md.drive(system)

system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)

Running OOMMF (TclOOMMFRunner) [2021/04/13 16:16]... (7.2 s)


Output()

Finally, we add a spin torque tranfer term brought by a spin polarized current on the x direction. We can see the formation of a vortex in the wider part of the plate.

In [17]:
ux = 400 # current speed in the x direction (m/s)
beta = 0.5 # non-abiabatic STT parameter

system.dynamics += mm.ZhangLi(u=ux, beta=beta)
td = oc.TimeDriver()
td.drive(system, t=0.2e-9, n=100)

system.m.z.plane('z').k3d_scalar(filter_field=system.m.norm)

Running OOMMF (TclOOMMFRunner) [2021/04/13 16:16]... (9.6 s)


Output()

This result is very encouraging for the generation of skyrmions and skyrmion lattices for spintronics applications ([Zhou, 2014](https://www.nature.com/articles/ncomms5652)). 

That completes the Ubermag demonstration. I hope you enjoyed it and can work with your own geometries soon.

*Victor*